# TCSS 588 Hands-on assignment #3
## Mengting Weng, Guang Xu, Wenfei Yin (group_WXY)

## 1.Experiment combinations of the above feature selection and classification methods and apply to the ADR data to predict side effects. Evaluate the performance using 10-fold cross validation, repeated 3 times. Note that you need to perform feature selection in each fold and each run of your cross validation results. In other words, you will perform feature selection and classification a total of 30 times

### Read Data:

In [1]:
source("https://bioconductor.org/biocLite.R")
biocLite(c("iterativeBMA", "class", "e1071"))
library(class)
library(e1071)
library (iterativeBMA)
## Read gene expression data
my.data <- read.table ("gene_expression_n438x978.txt", sep="\t", header=T)
dimnames(my.data)[[1]] <- my.data[,1]
my.data <- my.data[, -1]
dim (my.data)
my.data[1:5, 1:5]

Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.3 (2017-11-30).
Installing package(s) 'iterativeBMA', 'class', 'e1071'


package 'iterativeBMA' successfully unpacked and MD5 sums checked
package 'class' successfully unpacked and MD5 sums checked
package 'e1071' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\mengting\AppData\Local\Temp\RtmpIFbmiw\downloaded_packages


Old packages: 'BH', 'bindr', 'bindrcpp', 'bit', 'broom', 'callr', 'car',
  'caret', 'checkpoint', 'cowplot', 'curl', 'data.table', 'DBI', 'dbplyr',
  'ddalpha', 'dendextend', 'digest', 'doParallel', 'DRR', 'ellipse',
  'FactoMineR', 'forcats', 'foreach', 'foreign', 'fpc', 'ggpubr', 'ggrepel',
  'ggsci', 'glmnet', 'gridExtra', 'haven', 'hexbin', 'hms', 'htmlwidgets',
  'httpuv', 'iterators', 'jsonlite', 'kernlab', 'knitr', 'lava', 'lme4',
  'lubridate', 'maps', 'MASS', 'Matrix', 'mclust', 'mgcv', 'modelr', 'mvtnorm',
  'nlme', 'openssl', 'packrat', 'pbdZMQ', 'pillar', 'png', 'prodlim', 'psych',
  'quantmod', 'quantreg', 'R6', 'Rcpp', 'RcppEigen', 'RCurl', 'readxl',
  'recipes', 'repr', 'reprex', 'rlang', 'rmarkdown', 'robustbase', 'RODBC',
  'rpart', 'rprojroot', 'rsconnect', 'RSQLite', 'RUnit', 'scatterplot3d',
  'selectr', 'sfsmisc', 'sourcetools', 'stringi', 'stringr', 'survival',
  'tidyr', 'tidyselect', 'timeDate', 'TTR', 'utf8', 'viridis', 'viridisLite',
  'withr', 'xml2', 'xts', 

[1] 438 978

,PSME1,ATF1,RHEB,FOXO3,RHOA
ACETAZOLAMIDE,-0.015159099,-0.031470528,-0.004733488,0.02591061,0.0056296773
IRBESARTAN,-0.026811981,0.012151979,-0.025550148,-0.02401181,-0.0106717396
IPRATROPIUM BROMIDE,0.001017958,-0.008650622,-0.018128698,-0.02079971,-0.0002722781
EFAVIRENZ,-0.004398264,0.055387992,0.004658520,0.01380732,-0.0340697348
THIAMINE,0.001838965,-0.018079188,-0.011855532,-0.03705033,-0.0133954959


In [2]:
## Read ADR matrix
## Load gene expression data
adr.mat <- read.table ("ADRs_HLGT_n438x232.txt", sep="\t", header=T)
dimnames(adr.mat)[[1]] <- adr.mat [, 1]
adr.mat <- adr.mat[, -1]
dim (adr.mat)
adr.mat[1:5, 1:5]

[1] 438 232

,Abdominal.hernias.and.other.abdominal.wall.conditions,Abortions.and.stillbirth,Acid.base.disorders,Administration.site.reactions,Adrenal.gland.disorders
ACETAZOLAMIDE,0,0,1,1,0
IRBESARTAN,0,0,0,0,1
IPRATROPIUM BROMIDE,0,0,0,0,1
EFAVIRENZ,0,0,0,0,1
THIAMINE,0,0,0,0,0


### Combination 1 (high correlation and k-NN with k=10):¶

In [3]:
output <- matrix(nrow=232, ncol=8)
for (i in 1:232) {
    folds = cut(seq(1, nrow(my.data)), breaks=10, labels=FALSE)
    yDataRaw = adr.mat[i]
    yDataRaw = yDataRaw[rownames(my.data), , drop=FALSE]
    accuracies = c()
    ## Repeat for three times
    for (r in 1:3) {
        ## Feature selection
        cor.vector = abs(apply(my.data, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
        cor.col = names(cor.vector[cor.vector >= quantile(cor.vector, prob=1-5/100)])
        ## Shuffle Data
        xData = my.data[sample(nrow(my.data)), cor.col, drop=FALSE];
        yData = yDataRaw[rownames(xData), , drop=FALSE];
        predictions = c()
        for (k in 1:10) {
            testIndexes = which(folds == k, arr.ind=TRUE)
            xTestData = xData[testIndexes, , drop=FALSE]
            xTrainData = xData[-testIndexes, , drop=FALSE]
            yTrainData = yData[-testIndexes, , drop=FALSE]
            yTestData = yData[testIndexes, , drop=FALSE]
            results = as.numeric(knn(xTrainData, xTestData, as.numeric(unlist(yTrainData)), k = 10)) - 1

            predictions = c(predictions, results)
        }
        accuracy = mean(predictions == unlist(yData))
        accuracies = c(accuracies, accuracy)
    }
    accu.avg = mean(accuracies)
    output[i, 1] = accu.avg
}

In [4]:
output[, 1]

[1] 0.9680365 0.9771689 0.8105023 0.8242009 0.6240487 0.9337900 0.8614916
  [8] 0.5898021 0.8744292 0.7412481 0.8219178 0.8614916 0.7709285 0.6484018
 [15] 0.6286149 0.6088280 0.7777778 0.7534247 0.7526636 0.9589041 0.7937595
 [22] 0.7694064 0.9406393 0.7785388 0.7777778 0.6948250 0.9611872 0.9353120
 [29] 0.5928463 0.9566210 0.7458143 0.7047184 0.8234399 0.9589041 0.8995434
 [36] 0.7085236 0.9566210 0.9269406 0.8485540 0.6103501 0.8500761 0.7724505
 [43] 0.9246575 0.9360731 0.7488584 0.8713851 0.6019787 0.6270928 0.6248097
 [50] 0.8599696 0.5928463 0.7869102 0.7488584 0.8652968 0.9703196 0.8767123
 [57] 0.7016743 0.9589041 0.8523592 0.7397260 0.6811263 0.8949772 0.9337900
 [64] 0.7024353 0.9497717 0.8523592 0.9680365 0.7891933 0.7222222 0.6362253
 [71] 0.9474886 0.6293760 0.9360731 0.7397260 0.8325723 0.9566210 0.9406393
 [78] 0.7922374 0.8828006 0.6095890 0.7024353 0.7511416 0.6445967 0.9634703
 [85] 0.7732116 0.9634703 0.8150685 0.9840183 0.8051750 0.8949772 0.9406393
 [92] 0.6894977 0.8858447 0.6423135 0.9269406 0.7876712 0.9680365 0.7983257
 [99] 0.7724505 0.5913242 0.5890411 0.9680365 0.9680365 0.8538813 0.6674277
[106] 0.9634703 0.9429224 0.9566210 0.9018265 0.6864536 0.7808219 0.5700152
[113] 0.9680365 0.5745814 0.9132420 0.9444444 0.9657534 0.8059361 0.7016743
[120] 0.9748858 0.9452055 0.9246575 0.9634703 0.9124810 0.8401826 0.7092846
[127] 0.6461187 0.9566210 0.8957382 0.5989346 0.8774734 0.8873668 0.7663623
[134] 0.6377473 0.5844749 0.7640791 0.9634703 0.5859970 0.9657534 0.9497717
[141] 0.8455099 0.9680365 0.6133942 0.9680365 0.7884323 0.6156773 0.9048706
[148] 0.6057839 0.9680365 0.9018265 0.6476408 0.6202435 0.8576865 0.8340944
[155] 0.6217656 0.7077626 0.9200913 0.9680365 0.9063927 0.6171994 0.6628615
[162] 0.6423135 0.6484018 0.6225266 0.9033486 0.6019787 0.8698630 0.9474886
[169] 0.7602740 0.9041096 0.9406393 0.7488584 0.8462709 0.7374429 0.8340944
[176] 0.8082192 0.9337900 0.7564688 0.6171994 0.6194825 0.9041096 0.9566210
[183] 0.8607306 0.9611872 0.8554033 0.5936073 0.9611872 0.6225266 0.6369863
[190] 0.8181126 0.5920852 0.6461187 0.6255708 0.9497717 0.8181126 0.9589041
[197] 0.6925419 0.6925419 0.9337900 0.6347032 0.8554033 0.8546423 0.8904110
[204] 0.9383562 0.7975647 0.7937595 0.8926941 0.8911720 0.8812785 0.9292237
[211] 0.5768645 0.8394216 0.8554033 0.6453577 0.5776256 0.5791476 0.9634703
[218] 0.6407915 0.8805175 0.9048706 0.6757991 0.6598174 0.6149163 0.7298326
[225] 0.8226788 0.8926941 0.6042618 0.6149163 0.9086758 0.8447489 0.9330289
[232] 0.6742770

### Combination 2 (medium correlation and k-NN with k=10):

In [5]:
for (i in 1:232) {
    folds = cut(seq(1, nrow(my.data)), breaks=10, labels=FALSE)
    yDataRaw = adr.mat[i]
    yDataRaw = yDataRaw[rownames(my.data), , drop=FALSE]
    accuracies = c()
    ## Repeat for three times
    for (r in 1:3) {
        ## Feature selection
        cor.vector = abs(apply(my.data, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
        cor.col = names(cor.vector[cor.vector >= quantile(cor.vector, prob=1-10/100)])
        ## Shuffle Data
        xData = my.data[sample(nrow(my.data)), cor.col, drop=FALSE];
        yData = yDataRaw[rownames(xData), , drop=FALSE];
        predictions = c()
        for (k in 1:10) {
            testIndexes = which(folds == k, arr.ind=TRUE)
            xTestData = xData[testIndexes, , drop=FALSE]
            xTrainData = xData[-testIndexes, , drop=FALSE]
            yTrainData = yData[-testIndexes, , drop=FALSE]
            yTestData = yData[testIndexes, , drop=FALSE]
            results = as.numeric(knn(xTrainData, xTestData, as.numeric(unlist(yTrainData)), k = 10)) - 1

            predictions = c(predictions, results)
        }
        accuracy = mean(predictions == unlist(yData))
        accuracies = c(accuracies, accuracy)
    }
    accu.avg = mean(accuracies)
    output[i, 2] = accu.avg
}

In [6]:
output[, 2]

[1] 0.9680365 0.9771689 0.8150685 0.8249619 0.6681887 0.9337900 0.8652968
  [8] 0.5920852 0.8759513 0.7427702 0.8242009 0.8645358 0.7709285 0.6415525
 [15] 0.6308980 0.6171994 0.7564688 0.7572298 0.7587519 0.9589041 0.7846271
 [22] 0.7732116 0.9406393 0.7709285 0.7929985 0.6826484 0.9611872 0.9337900
 [29] 0.6133942 0.9566210 0.7503805 0.6712329 0.8196347 0.9589041 0.8995434
 [36] 0.6803653 0.9566210 0.9269406 0.8523592 0.5875190 0.8523592 0.7785388
 [43] 0.9246575 0.9360731 0.7473364 0.8706240 0.5745814 0.6042618 0.6179604
 [50] 0.8614916 0.6232877 0.7861492 0.7442922 0.8683409 0.9703196 0.8736682
 [57] 0.6445967 0.9589041 0.8538813 0.7328767 0.6826484 0.8964992 0.9337900
 [64] 0.6917808 0.9497717 0.8515982 0.9680365 0.7952816 0.7024353 0.6179604
 [71] 0.9474886 0.6560122 0.9360731 0.7389650 0.8310502 0.9566210 0.9414003
 [78] 0.7846271 0.8835616 0.6179604 0.6879756 0.7511416 0.6544901 0.9634703
 [85] 0.7876712 0.9634703 0.8158295 0.9840183 0.8105023 0.8957382 0.9406393
 [92] 0.6598174 0.8911720 0.6506849 0.9269406 0.7876712 0.9680365 0.8013699
 [99] 0.7656012 0.6027397 0.6019787 0.9680365 0.9680365 0.8523592 0.6240487
[106] 0.9634703 0.9429224 0.9566210 0.9018265 0.6659056 0.7838661 0.5646880
[113] 0.9680365 0.6019787 0.9117199 0.9429224 0.9657534 0.7998478 0.7184170
[120] 0.9748858 0.9452055 0.9246575 0.9634703 0.9124810 0.8394216 0.6879756
[127] 0.6331811 0.9566210 0.8995434 0.6735160 0.8767123 0.8858447 0.7686454
[134] 0.5966514 0.6103501 0.7549467 0.9634703 0.5943683 0.9657534 0.9497717
[141] 0.8523592 0.9680365 0.6232877 0.9680365 0.7945205 0.6095890 0.9033486
[148] 0.5768645 0.9680365 0.9018265 0.6210046 0.6035008 0.8538813 0.8378995
[155] 0.5859970 0.7024353 0.9200913 0.9680365 0.9063927 0.6141553 0.6484018
[162] 0.6232877 0.6621005 0.6111111 0.9041096 0.5768645 0.8691020 0.9474886
[169] 0.7404871 0.9041096 0.9406393 0.7488584 0.8455099 0.7260274 0.8348554
[176] 0.8013699 0.9337900 0.7678843 0.6445967 0.6362253 0.9041096 0.9566210
[183] 0.8614916 0.9611872 0.8569254 0.5943683 0.9611872 0.6248097 0.5692542
[190] 0.8188737 0.6156773 0.6628615 0.6141553 0.9497717 0.8059361 0.9589041
[197] 0.6834094 0.6347032 0.9337900 0.6598174 0.8584475 0.8592085 0.8904110
[204] 0.9383562 0.7876712 0.8089802 0.8926941 0.8881279 0.8812785 0.9292237
[211] 0.6225266 0.8401826 0.8554033 0.6445967 0.5913242 0.6225266 0.9634703
[218] 0.6818874 0.8782344 0.9041096 0.6750381 0.6423135 0.6552511 0.7207002
[225] 0.8318113 0.8934551 0.6210046 0.5821918 0.9086758 0.8447489 0.9315068
[232] 0.6522070

### Combination 3 (low correlation and k-NN with k=10):

In [7]:
for (i in 1:232) {
    folds = cut(seq(1, nrow(my.data)), breaks=10, labels=FALSE)
    yDataRaw = adr.mat[i]
    yDataRaw = yDataRaw[rownames(my.data), , drop=FALSE]
    accuracies = c()
    ## Repeat for three times
    for (r in 1:3) {
        ## Feature selection
        cor.vector = abs(apply(my.data, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
        cor.col = names(cor.vector[cor.vector >= quantile(cor.vector, prob=1-25/100)])
        ## Shuffle Data
        xData = my.data[sample(nrow(my.data)), cor.col, drop=FALSE];
        yData = yDataRaw[rownames(xData), , drop=FALSE];
        predictions = c()
        for (k in 1:10) {
            testIndexes = which(folds == k, arr.ind=TRUE)
            xTestData = xData[testIndexes, , drop=FALSE]
            xTrainData = xData[-testIndexes, , drop=FALSE]
            yTrainData = yData[-testIndexes, , drop=FALSE]
            yTestData = yData[testIndexes, , drop=FALSE]
            results = as.numeric(knn(xTrainData, xTestData, as.numeric(unlist(yTrainData)), k = 10)) - 1

            predictions = c(predictions, results)
        }
        accuracy = mean(predictions == unlist(yData))
        accuracies = c(accuracies, accuracy)
    }
    accu.avg = mean(accuracies)
    output[i, 3] = accu.avg
}

In [8]:
output[, 3]

[1] 0.9680365 0.9771689 0.8173516 0.8196347 0.6575342 0.9337900 0.8652968
  [8] 0.5806697 0.8736682 0.7229833 0.8242009 0.8652968 0.7724505 0.6324201
 [15] 0.6316591 0.5776256 0.7678843 0.7572298 0.7610350 0.9589041 0.7754947
 [22] 0.7511416 0.9383562 0.7732116 0.7861492 0.6811263 0.9611872 0.9337900
 [29] 0.6126332 0.9566210 0.7420091 0.6780822 0.8234399 0.9589041 0.8995434
 [36] 0.6917808 0.9566210 0.9269406 0.8515982 0.5722983 0.8500761 0.7648402
 [43] 0.9246575 0.9360731 0.7656012 0.8721461 0.5799087 0.6407915 0.5646880
 [50] 0.8592085 0.5715373 0.7831050 0.7397260 0.8683409 0.9703196 0.8767123
 [57] 0.6818874 0.9589041 0.8554033 0.7313546 0.6582953 0.8949772 0.9337900
 [64] 0.7039574 0.9497717 0.8515982 0.9680365 0.7914764 0.6917808 0.5806697
 [71] 0.9474886 0.6377473 0.9360731 0.7207002 0.8302892 0.9566210 0.9391172
 [78] 0.7922374 0.8835616 0.6179604 0.6826484 0.7412481 0.6362253 0.9634703
 [85] 0.7739726 0.9634703 0.8165906 0.9840183 0.8105023 0.8949772 0.9406393
 [92] 0.6529680 0.8904110 0.6438356 0.9269406 0.7861492 0.9680365 0.7945205
 [99] 0.7709285 0.5570776 0.5814307 0.9680365 0.9680365 0.8515982 0.6027397
[106] 0.9634703 0.9429224 0.9566210 0.9018265 0.6704718 0.7792998 0.5844749
[113] 0.9680365 0.5646880 0.9132420 0.9429224 0.9657534 0.8028919 0.6993912
[120] 0.9748858 0.9452055 0.9246575 0.9634703 0.9086758 0.8401826 0.7191781
[127] 0.5928463 0.9566210 0.8972603 0.6369863 0.8767123 0.8835616 0.7625571
[134] 0.5981735 0.5662100 0.7519026 0.9634703 0.5783866 0.9657534 0.9497717
[141] 0.8508371 0.9680365 0.6301370 0.9680365 0.7907154 0.5844749 0.9033486
[148] 0.5509893 0.9680365 0.9018265 0.6468798 0.5791476 0.8470320 0.8386606
[155] 0.5502283 0.7009132 0.9200913 0.9680365 0.9063927 0.6362253 0.6194825
[162] 0.5783866 0.6369863 0.5867580 0.9041096 0.5859970 0.8691020 0.9474886
[169] 0.7108067 0.9041096 0.9406393 0.7435312 0.8447489 0.7153729 0.8325723
[176] 0.8021309 0.9337900 0.7587519 0.6019787 0.5791476 0.9041096 0.9566210
[183] 0.8607306 0.9611872 0.8554033 0.5966514 0.9611872 0.5791476 0.5761035
[190] 0.8097412 0.5890411 0.6407915 0.5578387 0.9497717 0.8127854 0.9589041
[197] 0.6841705 0.7024353 0.9337900 0.6225266 0.8546423 0.8554033 0.8881279
[204] 0.9383562 0.7739726 0.7884323 0.8926941 0.8881279 0.8812785 0.9292237
[211] 0.5684932 0.8325723 0.8523592 0.6027397 0.5890411 0.5920852 0.9634703
[218] 0.6575342 0.8789954 0.9041096 0.6445967 0.6225266 0.6468798 0.7031963
[225] 0.8219178 0.8926941 0.5578387 0.6027397 0.9086758 0.8439878 0.9315068
[232] 0.6187215

### Combination 4 (none and k-NN with k=10):

In [9]:
for (i in 1:232) {
    folds = cut(seq(1, nrow(my.data)), breaks=10, labels=FALSE)
    yDataRaw = adr.mat[i]
    yDataRaw = yDataRaw[rownames(my.data), , drop=FALSE]
    accuracies = c()
    ## Repeat for three times
    for (r in 1:3) {
        ## Shuffle Data
        xData = my.data[sample(nrow(my.data)), , drop=FALSE];
        yData = yDataRaw[rownames(xData), , drop=FALSE];
        predictions = c()
        for (k in 1:10) {
            testIndexes = which(folds == k, arr.ind=TRUE)
            xTestData = xData[testIndexes, , drop=FALSE]
            xTrainData = xData[-testIndexes, , drop=FALSE]
            yTrainData = yData[-testIndexes, , drop=FALSE]
            yTestData = yData[testIndexes, , drop=FALSE]
            results = as.numeric(knn(xTrainData, xTestData, as.numeric(unlist(yTrainData)), k = 10)) - 1

            predictions = c(predictions, results)
        }
        accuracy = mean(predictions == unlist(yData))
        accuracies = c(accuracies, accuracy)
    }
    accu.avg = mean(accuracies)
    output[i, 4] = accu.avg
}

In [10]:
output[, 4]

[1] 0.9680365 0.9771689 0.8112633 0.8165906 0.5570776 0.9337900 0.8645358
  [8] 0.5334855 0.8706240 0.6872146 0.8242009 0.8630137 0.7633181 0.5388128
 [15] 0.5783866 0.4802131 0.7480974 0.7267884 0.7404871 0.9589041 0.7625571
 [22] 0.7671233 0.9406393 0.7511416 0.7694064 0.6225266 0.9611872 0.9337900
 [29] 0.5129376 0.9566210 0.7229833 0.6217656 0.8112633 0.9589041 0.8995434
 [36] 0.6438356 0.9566210 0.9269406 0.8439878 0.5372907 0.8462709 0.7480974
 [43] 0.9246575 0.9360731 0.7397260 0.8721461 0.5380518 0.5175038 0.5000000
 [50] 0.8508371 0.4832572 0.7602740 0.7070015 0.8652968 0.9703196 0.8729072
 [57] 0.6499239 0.9589041 0.8455099 0.7100457 0.6308980 0.8926941 0.9337900
 [64] 0.6560122 0.9497717 0.8493151 0.9680365 0.7701674 0.6719939 0.5273973
 [71] 0.9474886 0.5525114 0.9360731 0.6940639 0.8226788 0.9566210 0.9406393
 [78] 0.7831050 0.8835616 0.5350076 0.6499239 0.7351598 0.5547945 0.9634703
 [85] 0.7496195 0.9634703 0.8089802 0.9840183 0.7983257 0.8942161 0.9406393
 [92] 0.5974125 0.8919330 0.5776256 0.9269406 0.7785388 0.9680365 0.7846271
 [99] 0.7595129 0.4939117 0.4885845 0.9680365 0.9680365 0.8493151 0.5677321
[106] 0.9634703 0.9429224 0.9566210 0.8995434 0.6187215 0.7519026 0.5121766
[113] 0.9680365 0.4923896 0.9109589 0.9429224 0.9657534 0.7968037 0.6491629
[120] 0.9748858 0.9452055 0.9246575 0.9634703 0.9086758 0.8386606 0.6659056
[127] 0.5449011 0.9566210 0.8972603 0.5479452 0.8759513 0.8812785 0.7526636
[134] 0.5235921 0.4634703 0.7252664 0.9634703 0.4680365 0.9657534 0.9497717
[141] 0.8485540 0.9680365 0.5966514 0.9680365 0.7846271 0.5228311 0.9041096
[148] 0.4885845 0.9680365 0.9010654 0.5327245 0.5228311 0.8477930 0.8348554
[155] 0.4824962 0.6415525 0.9193303 0.9680365 0.9063927 0.5639269 0.6248097
[162] 0.5464231 0.6035008 0.4863014 0.9010654 0.5258752 0.8660578 0.9474886
[169] 0.7130898 0.9041096 0.9406393 0.7351598 0.8409437 0.6765601 0.8310502
[176] 0.7983257 0.9337900 0.7267884 0.4733638 0.4855403 0.9003044 0.9566210
[183] 0.8607306 0.9611872 0.8500761 0.4961948 0.9611872 0.4718417 0.4984779
[190] 0.8074581 0.5015221 0.5593607 0.4802131 0.9497717 0.8028919 0.9589041
[197] 0.6225266 0.6499239 0.9337900 0.5654490 0.8500761 0.8485540 0.8873668
[204] 0.9383562 0.7549467 0.7914764 0.8926941 0.8873668 0.8797565 0.9284627
[211] 0.5015221 0.8325723 0.8500761 0.5646880 0.5403349 0.5251142 0.9634703
[218] 0.5981735 0.8782344 0.9025875 0.5989346 0.6118721 0.5433790 0.6910198
[225] 0.8249619 0.8926941 0.4855403 0.5327245 0.9086758 0.8401826 0.9307458
[232] 0.5403349

### Combination 5 (medium correlation and SVM):

In [11]:
for (i in 1:232) {
    folds = cut(seq(1, nrow(my.data)), breaks=10, labels=FALSE)
    yDataRaw = adr.mat[i]
    yDataRaw = yDataRaw[rownames(my.data), , drop=FALSE]
    accuracies = c()
    ## Repeat for three times
    for (r in 1:3) {
        ## Feature selection
        cor.vector = abs(apply(my.data, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
        cor.col = names(cor.vector[cor.vector >= quantile(cor.vector, prob=1-10/100)])
        ## Shuffle Data
        xData = my.data[sample(nrow(my.data)), cor.col, drop=FALSE];
        yData = yDataRaw[rownames(xData), , drop=FALSE];
        predictions = c()
        for (k in 1:10) {
            testIndexes = which(folds == k, arr.ind=TRUE)
            xTestData = xData[testIndexes, , drop=FALSE]
            xTrainData = xData[-testIndexes, , drop=FALSE]
            yTrainData = yData[-testIndexes, , drop=FALSE]
            yTestData = yData[testIndexes, , drop=FALSE]
            model = svm(xTrainData, yTrainData)
            prediction = round(as.numeric(predict(model, xTestData)))
            predictions = c(predictions, prediction)
        }
        accuracy = mean(predictions == unlist(yData))
        accuracies = c(accuracies, accuracy)
    }
    accu.avg = mean(accuracies)
    output[i, 5] = accu.avg
}

In [12]:
output

0.9680365,0.9680365,0.9680365,0.9680365,0.9680365,NA,NA,NA
0.9771689,0.9771689,0.9771689,0.9771689,0.9771689,NA,NA,NA
0.8105023,0.8150685,0.8173516,0.8112633,0.8127854,NA,NA,NA
0.8242009,0.8249619,0.8196347,0.8165906,0.8219178,NA,NA,NA
0.6240487,0.6681887,0.6575342,0.5570776,0.7366819,NA,NA,NA
0.9337900,0.9337900,0.9337900,0.9337900,0.9337900,NA,NA,NA
0.8614916,0.8652968,0.8652968,0.8645358,0.8652968,NA,NA,NA
0.5898021,0.5920852,0.5806697,0.5334855,0.6735160,NA,NA,NA
0.8744292,0.8759513,0.8736682,0.8706240,0.8744292,NA,NA,NA
0.7412481,0.7427702,0.7229833,0.6872146,0.7382040,NA,NA,NA
0.8219178,0.8242009,0.8242009,0.8242009,0.8264840,NA,NA,NA


### Combination 6 (high correlation and SVM):

In [14]:
for (i in 1:232) {
    folds = cut(seq(1, nrow(my.data)), breaks=10, labels=FALSE)
    yDataRaw = adr.mat[i]
    yDataRaw = yDataRaw[rownames(my.data), , drop=FALSE]
    accuracies = c()
    ## Repeat for three times
    for (r in 1:3) {
        ## Feature selection
        cor.vector = abs(apply(my.data, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
        cor.col = names(cor.vector[cor.vector >= quantile(cor.vector, prob=1-5/100)])
        ## Shuffle Data
        xData = my.data[sample(nrow(my.data)), cor.col, drop=FALSE];
        yData = yDataRaw[rownames(xData), , drop=FALSE];
        predictions = c()
        for (k in 1:10) {
            testIndexes = which(folds == k, arr.ind=TRUE)
            xTestData = xData[testIndexes, , drop=FALSE]
            xTrainData = xData[-testIndexes, , drop=FALSE]
            yTrainData = yData[-testIndexes, , drop=FALSE]
            yTestData = yData[testIndexes, , drop=FALSE]
            model = svm(xTrainData, yTrainData)
            prediction = round(as.numeric(predict(model, xTestData)))
            predictions = c(predictions, prediction)
        }
        accuracy = mean(predictions == unlist(yData))
        accuracies = c(accuracies, accuracy)
    }
    accu.avg = mean(accuracies)
    output[i, 6] = accu.avg
}

In [15]:
output

0.9680365,0.9680365,0.9680365,0.9680365,0.9680365,0.9680365,NA,NA
0.9771689,0.9771689,0.9771689,0.9771689,0.9771689,0.9771689,NA,NA
0.8105023,0.8150685,0.8173516,0.8112633,0.8127854,0.8143075,NA,NA
0.8242009,0.8249619,0.8196347,0.8165906,0.8219178,0.8219178,NA,NA
0.6240487,0.6681887,0.6575342,0.5570776,0.7366819,0.7123288,NA,NA
0.9337900,0.9337900,0.9337900,0.9337900,0.9337900,0.9337900,NA,NA
0.8614916,0.8652968,0.8652968,0.8645358,0.8652968,0.8652968,NA,NA
0.5898021,0.5920852,0.5806697,0.5334855,0.6735160,0.6651446,NA,NA
0.8744292,0.8759513,0.8736682,0.8706240,0.8744292,0.8744292,NA,NA
0.7412481,0.7427702,0.7229833,0.6872146,0.7382040,0.7465753,NA,NA
0.8219178,0.8242009,0.8242009,0.8242009,0.8264840,0.8264840,NA,NA


### Combination 7 (medium correlation and BMA):

In [24]:
for (i in 1:232) {
    folds = cut(seq(1, nrow(my.data)), breaks=10, labels=FALSE)
    yDataRaw = adr.mat[i]
    yDataRaw = yDataRaw[rownames(my.data), , drop=FALSE]
    accuracies = c()
    ## Repeat for three times
    for (r in 1:3) {
        ## Feature selection
        cor.vector = abs(apply(my.data, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
        cor.col = names(cor.vector[cor.vector >= quantile(cor.vector, prob=1-10/100)])
        ## Shuffle Data
        xData = my.data[sample(nrow(my.data)), cor.col, drop=FALSE];
        yData = yDataRaw[rownames(xData), , drop=FALSE];
        predictions = c()
        for (k in 1:10) {
            testIndexes = which(folds == k, arr.ind=TRUE)
            xTestData = xData[testIndexes, , drop=FALSE]
            xTrainData = xData[-testIndexes, , drop=FALSE]
            yTrainData = yData[-testIndexes, , drop=FALSE]
            yTestData = yData[testIndexes, , drop=FALSE]
            prediction <- iterateBMAglm.train.predict (train.expr.set=ExpressionSet(assayData=xTrainData), test.expr.set=ExpressionSet(assayData=xTestData), train.class=yTrainData)
            prediction = round(as.numeric(prediction))
            predictions = c(predictions, prediction)
        }
        accuracy = mean(predictions == unlist(yData))
        accuracies = c(accuracies, accuracy)
    }
    accu.avg = mean(accuracies)
    output[i, 7] = accu.avg
}

ERROR: Error in (function (classes, fdef, mtable) : unable to find an inherited method for function 'ExpressionSet' for signature '"data.frame"'


In [19]:
help(iterateBMAglm.train.predict)

### Combination 8 (high correlation and BMA):

In [ ]:
for (i in 1:232) {
    folds = cut(seq(1, nrow(my.data)), breaks=10, labels=FALSE)
    yDataRaw = adr.mat[i]
    yDataRaw = yDataRaw[rownames(my.data), , drop=FALSE]
    accuracies = c()
    ## Repeat for three times
    for (r in 1:3) {
        ## Feature selection
        cor.vector = abs(apply(my.data, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
        cor.col = names(cor.vector[cor.vector >= quantile(cor.vector, prob=1-10/100)])
        ## Shuffle Data
        xData = my.data[sample(nrow(my.data)), cor.col, drop=FALSE];
        yData = yDataRaw[rownames(xData), , drop=FALSE];
        predictions = c()
        for (k in 1:10) {
            testIndexes = which(folds == k, arr.ind=TRUE)
            xTestData = xData[testIndexes, , drop=FALSE]
            xTrainData = xData[-testIndexes, , drop=FALSE]
            yTrainData = yData[-testIndexes, , drop=FALSE]
            yTestData = yData[testIndexes, , drop=FALSE]
            prediction <- iterateBMAglm.train.predict (train.expr.set=ExpressionSet (assayData=xTrainData), test.expr.set=ExpressionSet(assayData=xTestData), train.class=yTrainData)
            prediction = round(as.numeric(prediction)))
            predictions = c(predictions, prediction)
        }
        accuracy = mean(predictions == unlist(yData))
        accuracies = c(accuracies, accuracy)
    }
    accu.avg = mean(accuracies)
    output[i, 8] = accu.avg
}

In [ ]:
output

### Output average prediction accuracy


In [33]:
methodNames = c("high-correlation-knn-10", "medium-correlation-knn-10", "low-correlation-knn-10", "none-knn-10", "meidum-correlation-svm", "high-correlation-svm", "meidum-correlation-bma", "high-correlatio-bma")
colnames(output) = methodNames
rownames(output) = colnames(adr.mat)
write.csv(output, file="output1.csv")
#write.table(output,file='output.csv',sep="\t")

## 2. Compare the prediction accuracy of the methods you tried in your report. In particular, address the following questions:

### a.Which side effects you can predict with the highest accuracy in each combination?

In [39]:
for (i in 1:8) {
    print(which(output[,i] == max(output[,i])))
}



Gastrointestinal.tract.disorders.congenital 
                                         88 
Gastrointestinal.tract.disorders.congenital 
                                         88 
Gastrointestinal.tract.disorders.congenital 
                                         88 
Gastrointestinal.tract.disorders.congenital 
                                         88 
Gastrointestinal.tract.disorders.congenital 
                                         88 
Gastrointestinal.tract.disorders.congenital 
                                         88 
named integer(0)
named integer(0)


### b.Which side effects you predict with the lowest accuracy in each combination?

In [40]:
for (i in 1:8) {
    print(which(output[,i] == min(output[,i])))
}

Inner.ear.and.VIIIth.cranial.nerve.disorders 
                                         112 
Inner.ear.and.VIIIth.cranial.nerve.disorders 
                                         112 
Oral.soft.tissue.conditions 
                        155 
Movement.disorders..incl.parkinsonism. 
                                   135 
Anaemias.nonhaemolytic.and.marrow.depression 
                                           8 
Inner.ear.and.VIIIth.cranial.nerve.disorders 
                                         112 
named integer(0)
named integer(0)


### c.Some side effects have unbalanced class sizes. Did you do anything about that? Why? If so, is your method effective?

### d.Which feature selection and/or classification method would you consider as the “winner” in your empirical study? You can include the results from HW1 in your discussion.

### f.Any interesting negative results?